# Data Extraction and Web Scraping

We will use a real-world example to demonstrate the concepts of data extraction via REST API and GraphQL. The APIs we will be using are the [GitHub REST API](https://docs.github.com/en/rest) and [GitHub GraphQL API](https://docs.github.com/en/graphql).

They are both APIs provided by GitHub to access data on their platform. The GraphQL API is a newer API. You can access data such as user profiles, repositories, issues, pull requests, releases etc. from GitHub.

We will use the following libraries for this lesson:

- `requests` for making HTTP requests
- `json` for parsing JSON responses
- `pandas` and `numpy` for data manipulation
- `sqlalchemy` for writing data to database (DuckDB)
- `beautifulsoup4` for web scraping

## REST API

Usually, before we can make a request to an API, we need to register for an API key. This is for the API provider to track usage, prevent abuse and to authenticate users. For more information on how to authenticate with GitHub API, refer to the [GitHub API docs](https://docs.github.com/en/rest/overview/resources-in-the-rest-api?apiVersion=2022-11-28#authentication). 

For this lesson, you will need to a personal access token (API key). If you do not have an API key, you can create a new one by folllowing the instructions in the official GitHub documentation [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#creating-a-fine-grained-personal-access-token).

We will be retrieving data from some popular repositories on GitHub.

### Setting up Github API

After getting Github API key, please create a file called `.env` at the root folder of this repository.

Next, open the `.env` file and enter the follow:

```text
GITHUB_TOKEN = '<YOUR-GITHUB-ACCESS-TOKEN>'
```

Save and close the file when done.

> Please note that when you sync your work to Github, this file will not be synced. Please keep a copy of `.env` safe elsewhere.

In [ ]:
import requests
import os
from dotenv import load_dotenv

# if you got module not found error for dotenv please run the following command
# `conda activate bde`
# `conda install -c conda-forge python-dotenv`

In [ ]:
load_dotenv()

In [ ]:
# Please input your GitHub personal access token here
access_token = os.getenv('GITHUB_TOKEN')

Github API endpoints are in the format 
```
https://api.github.com/repos/{owner}/{repo}
```
and appended with the resource you want to access.

For example, to access the `issues` resource, the endpoint is
```
https://api.github.com/repos/{owner}/{repo}/issues
```

---

Here are some common endpoints:

| Resource | Endpoint |
| --- | --- |
| Issues | `/issues` |
| Pull Requests | `/pulls` |
| Commits | `/commits` |
| Contributors | `/contributors` |
| Languages | `/languages` |
| Releases | `/releases` |
| Tags | `/tags` |
| Branches | `/branches` |
| Forks | `/forks` |
| Stargazers | `/stargazers` |
| Subscribers | `/subscribers` |
| Subscription | `/subscription` |

Here are the HTTP verbs for the Github API:

| Verb | Description |
| --- | --- |
| GET | Used for retrieving resources. |
| POST | Used for creating resources. |
| PATCH | Used for updating resources with partial JSON data. For instance, an Issue resource has title and body attributes. A PATCH request may accept one or more of the attributes to update the resource. |
| PUT | Used for replacing resources or collections. For PUT requests with no body attribute, be sure to set the Content-Length header to zero. |
| DELETE | Used for deleting resources. |

We will only be using the GET verb for this project, 


`Pandas` is an open-source library, its code repository is hosted on GitHub. You can access the repository at [https://github.com/pandas-dev/pandas](https://github.com/pandas-dev/pandas).

Let's try to extract the past `releases` (library versions) from the the repository. The API endpoint is `https://api.github.com/repos/pandas-dev/pandas/releases`.

The access token needs to be passed in the `Authorization` header. The token is a personal access token (PAT) which you can generate on GitHub. The token is used to authenticate you as a user and to authorize you to access the repository.

It is also recommended to pass in the following headers:

- `Accept: application/vnd.github+json` - to specify the preferred format of the response
- `X-GitHub-Api-Version:2022-11-28` - to specify the version of the API to use

---

If we use a `GET` request to access the endpoint, we will get a list of releases in JSON format (default limit of _30 results_ per page). More info on the `releases` resource [here](https://docs.github.com/en/rest/releases/releases?apiVersion=2022-11-28).

In [ ]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/releases", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer {access_token}"})

We can check if the response is successful by checking the status code of the response. If the status code is 200, then the response is successful.

In [ ]:
response.status_code

Use the `.json()` method to return the JSON format of the response object.

In [ ]:
releases_page_1 = response.json()

In [ ]:
type(releases_page_1)

In [ ]:
len(releases_page_1)

In [ ]:
releases_page_1[0]

In [ ]:
releases_page_1[0]['name']

In [ ]:
releases_page_1[0]['published_at']

The latest version of `pandas` is the one shown above. Hence, the list is sorted from most recent to oldest.

While the last release on the list is:

In [ ]:
releases_page_1[-1]['name']

In [ ]:
releases_page_1[-1]['published_at']

Remember the default results per page is `30`. You can increase the number of results per page to a maximum of `100` by using the `per_page` query parameter.

In [ ]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/releases?per_page=100", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer {access_token}"})

In [ ]:
releases_page_1_100 = response.json()

In [ ]:
len(releases_page_1_100)

In [ ]:
releases_page_1_100[-1]

The default page is `1`, you can get the 2nd page by passing the `page` query parameter.

In [ ]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/releases?per_page=100&page=2", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer {access_token}"})

In [ ]:
releases_page_2_100 = response.json()

In [ ]:
len(releases_page_2_100)


We're interested in the following fields and want to save them into a dataframe:

- `tag_name`: The version name/number of the release.
- `published_at`: The date and time when the release was published.
- `body`: The release notes.

In [ ]:
import pandas as pd

In [ ]:
releases_with_essential_fields = [{"version": release["tag_name"], "published_at": release["published_at"], "summary": release["body"]} for release in releases_page_1_100]

In [ ]:
pandas_releases = pd.DataFrame(releases_with_essential_fields)

pandas_releases

In [ ]:
# convert `published_at` to the correct datetime format
pandas_releases['published_at'] = pd.to_datetime(pandas_releases['published_at'])

> What is the average interval (duration or differences in time) for pandas' releases (in no of days)?
>
> Hint 1: Sort the dataframe by `published_at` in ascending order.
> 
> Hint 2: Use [diff()](https://pandas.pydata.org/docs/reference/api/pandas.Series.diff.html) to calculate the time deltas between each `published_at`.
>
> Refer to [Time deltas](https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html#time-deltas) for more info.

> Print the `summary` of the first (1.0.0) and second (2.0.0) major releases of pandas.

Let's write the dataframe to a table in DuckDB.

First, we need to import the required libraries and get the absolute path of the project.

In [ ]:
import sqlalchemy as sqla
import os 

parent_dir = os.path.abspath(os.path.pardir)

In [ ]:
engine = sqla.create_engine(f'duckdb:///{parent_dir}/output/unit-2-3.db')

In [ ]:
pandas_releases.to_sql("pandas_releases", engine)

In [ ]:
engine.dispose()

Let's use another resource (endpoint) now. There are many active contributors to the pandas library, and suppose we want to know how many commits each contributor has contributed. We can retrieve the data using the `stats/contributors` endpoint.

More info on the endpoint can be found [here](https://docs.github.com/en/rest/metrics/statistics?apiVersion=2022-11-28).

In [ ]:
response = requests.get("https://api.github.com/repos/pandas-dev/pandas/stats/contributors", 
                        headers={"Accept": "application/vnd.github+json", "Authorization": f"Bearer {access_token}"})

In [ ]:
response.status_code

In [ ]:
contributors = response.json()

In [ ]:
type(contributors)

In [ ]:
len(contributors)

In [ ]:
contributors[0]

In [ ]:
contributors[0]["author"]

> Get the `total` and `login` (under `author`) fields and insert them into a dataframe. Then find out who has the most number of commits.
>
> Then, write the dataframe into a table named `pandas_contributors_commits` in duckdb.

## GraphQL

Github offers a new way to access its data, which is the GraphQL API. You can read more about it here: https://docs.github.com/graphql

GraphQL is introspetive, which means you can query the schema of the API itself. This is very useful when you are trying to figure out what data you can access. 

In GraphQL, to access the schema you can run the following query:

```graphql
{
  __schema {
    types {
      name
      kind
      description
    }
  }
}
```

Or to get the fields of a specific type (for example `"Release"`), you can run the following query:

```graphql
{
  __type(name: "Release") {
    name
    kind
    description
    fields {
      name
      description
    }
  }
}
```

It should return the following results (in JSON):

```json
{
  "data": {
    "__type": {
      "name": "Release",
      "kind": "OBJECT",
      "description": "A release contains the content for a release.",
      "fields": [
        {
          "name": "author",
          "description": "The author of the release"
        },
        {
          "name": "createdAt",
          "description": "Identifies the date and time when the object was created."
        },
        {
          "name": "databaseId",
          "description": "Identifies the primary key from the database."
        },
        {
          "name": "description",
          "description": "The description of the release."
        },
        {
          "name": "descriptionHTML",
          "description": "The description of this release rendered to HTML."
        },
        {
          "name": "id",
          "description": null
        },
        {
          "name": "isDraft",
          "description": "Whether or not the release is a draft"
        },
        {
          "name": "isLatest",
          "description": "Whether or not the release is the latest releast"
        },
        {
          "name": "isPrerelease",
          "description": "Whether or not the release is a prerelease"
        },
        {
          "name": "mentions",
          "description": "A list of users mentioned in the release description"
        },
        {
          "name": "name",
          "description": "The title of the release."
        },
        {
          "name": "publishedAt",
          "description": "Identifies the date and time when the release was created."
        },
        {
          "name": "reactionGroups",
          "description": "A list of reactions grouped by content left on the subject."
        },
        {
          "name": "reactions",
          "description": "A list of Reactions left on the Issue."
        },
        {
          "name": "releaseAssets",
          "description": "List of releases assets which are dependent on this release."
        },
        {
          "name": "repository",
          "description": "The repository that the release belongs to."
        },
        {
          "name": "resourcePath",
          "description": "The HTTP path for this issue"
        },
        {
          "name": "shortDescriptionHTML",
          "description": "A description of the release, rendered to HTML without any links in it."
        },
        {
          "name": "tag",
          "description": "The Git tag the release points to"
        },
        {
          "name": "tagCommit",
          "description": "The tag commit for this release."
        },
        {
          "name": "tagName",
          "description": "The name of the release's Git tag"
        },
        {
          "name": "updatedAt",
          "description": "Identifies the date and time when the object was last updated."
        },
        {
          "name": "url",
          "description": "The HTTP URL for this issue"
        },
        {
          "name": "viewerCanReact",
          "description": "Can user react to this subject"
        }
      ]
    }
  }
}
```

Follow the guide [here](https://docs.github.com/en/graphql/guides/introduction-to-graphql#discovering-the-graphql-api) for more info.

In [ ]:
# Feel free to test the query below - below query is to access the schema
query = """
query {
  __schema {
    types {
      name
      kind
      description
    }
  }
}
"""
            
response = requests.post("https://api.github.com/graphql", 
                        headers={"Authorization": f"Bearer {access_token}"}, json={"query": query})
response.json()

In [ ]:
# Feel free to test the query below - below query is to get the fields of a specific type (for example `"Release"`)
query = """
query {
  __type(name: "Release") {
    name
    kind
    description
    fields {
      name
      description
    }
  }
}
"""
            
response = requests.post("https://api.github.com/graphql", 
                        headers={"Authorization": f"Bearer {access_token}"}, json={"query": query})
response.json()

You can also get the schema via a `GET` request to the endpoint `https://api.github.com/graphql`:

In [ ]:
response = requests.get("https://api.github.com/graphql", 
                        headers={"Authorization": f"Bearer {access_token}"})

In [ ]:
schemas = response.json()

In [ ]:
schemas['data']['__schema'].keys()

In [ ]:
len(schemas['data']['__schema']['types'])

In [ ]:
schemas['data']['__schema']['types'][0]

It returns the same result as the `__schema` query above.

The GraphQL API only has a single endpoint:

```
https://api.github.com/graphql
```

In REST, HTTP verbs determine the operation performed. In GraphQL, you'll provide a JSON-encoded body whether you're performing a `query` or a `mutation`, so the HTTP verb is `POST`. The exception is an introspection query, which is a simple `GET` to the endpoint (which we did above).

To query GraphQL via `requests`, make a POST request with a JSON payload. The payload must contain a string called `query`.

GraphQL queries return only the data you specify. To form a query, you must specify fields within fields (also known as nested subfields) until you return only scalars.

For example, let's replicate the REST API exercise above where we extract the `releases`. You can see the resources in GraphQL which you can access [here](https://docs.github.com/en/graphql/reference/objects) - search for `releases`. 

We must specify the fields we want to extract:

In [ ]:
query = """
query {
    repository(owner: "pandas-dev", name: "pandas") {
        releases(first: 100) {
            totalCount
            edges {
                node {
                    tagName
                    description
                    publishedAt
                }
            }
        }
    }
}
"""
            

response = requests.post("https://api.github.com/graphql", 
                        headers={"Authorization": f"Bearer {access_token}"}, json={"query": query})

Looking at the composition line by line:

`query {`

Because we want to read data from the server, not modify it, query is the root operation. (If you don't specify an operation, query is also the default.)

`repository(owner:"pandas-dev", name:"pandas") {`

To begin the query, we want to find a repository object. The schema validation indicates this object requires an owner and a name argument.

`releases(first: 100) {`

To account for all releases in the repository, we call the releases object. (We could query a single release on a repository, but that would require us to know the tagName of the release we want to return and provide it as an argument.)

Some details about the releases object:

The docs tell us this object has the type `ReleaseConnection`. Schema validation indicates this object requires a last or first number of results as an argument, so we provide first 100. You can find more information about the `ReleaseConnection` type [here](https://docs.github.com/en/graphql/reference/objects#releaseconnection).

- `totalCount`

    The beauty of GraphQL is that we can retrieve the totalCount of the releases object by simply adding it to the query and it will be returned.

- `edges {`

    We know releases is a connection because it has the ReleaseConnection type. To retrieve data about individual release, we have to access the node via edges.

- `nodes {`

    Here we retrieve the nodes at the end of the edge. The ReleaseConnection docs indicate the nodes at the end of the ReleaseConnection type is a Release object.

    Now that we know we're retrieving a Release object, we can look at the docs and specify the fields we want to return:
    ```
    tagName
    description
    publishedAt
    ```

    Here we specify the tagName, description, and publishedAt as before.

In [ ]:
releases = response.json()

In [ ]:
type(releases)

In [ ]:
releases

In [ ]:
releases['data']['repository']['releases']['totalCount']

In [ ]:
releases['data']['repository']['releases']['edges'][0]['node']

In [ ]:
releases['data']['repository']['releases']['edges'][-1]['node']

> Query the `issues` resource, return the first 50 issues with the fields- `title`, `createdAt` and `author`, under the `author` field, return the `login` field.
>
> Refer to this [link](https://docs.github.com/en/graphql/reference/objects#issue) for more info.

## Web Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
r = requests.get("https://www.scrapethissite.com/pages/simple/")

In [ ]:
r.status_code

In [ ]:
type(r.text)

In [ ]:
print(r.text)

We will use BeautifulSoup to parse the HTML above. There are many parsers available. The default parser is `html.parser`, it does not require any additional installation. However, its performance is not the best. If you want better performance, you can use `lxml` parser. Refer to [here](https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=get#installing-a-parser) for more info.

In [ ]:
soup = BeautifulSoup(r.text, "html.parser")

We can look for all paragraphs in the page using the `find_all` method. This returns a list of all the paragraph `Tag`s in the page.

Refer to the [documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#bs4.Tag) for more details.

In [ ]:
paragraphs = soup.find_all('p')

In [ ]:
len(paragraphs)

In [ ]:
type(paragraphs[0])

In [ ]:
paragraphs[0]

In [ ]:
# get the actual text
paragraphs[0].text

We can find all the country names in the HTML by specifying the tag and class name. It is always good to be as specific as possible when selecting elements. This is because the more specific you are, the less likely you will select other unintended elements.

In [ ]:
country_name_elements = soup.find_all('h3', 'country-name')

or equivalently

`soup.find_all('h3', class_='country-name')`

In [ ]:
country_name_elements

> Extract just the country names string into a new list called `country_names`. Remove all whitespaces.
>
> Your results should be starting with `['Andorra', 'United Arab Emirates', 'Afghanistan', 'Antigua and Barbuda', ...]`

> Extract the capitals into a new list called `capital_names`.

It is common to collect all the required information at one go. For each country element, you can further use the `find` method to extract the child element.

Let's collect the capital, population and area for each country.

In [ ]:
countries = []
for country in soup.find_all('div', 'col-md-4 country'):
    country_info = {}
    country_info['name'] = country.find('h3').text.strip()
    country_info['capital'] = country.find('span', 'country-capital').text
    country_info['population'] = country.find('span', 'country-population').text
    country_info['area_km2'] = country.find('span', 'country-area').text
    countries.append(country_info)

In [ ]:
countries

Let's scrape a more complex example. This page has a table with pagination. We will scrape the rows in all the pages and store the data in a dataframe.

[https://www.scrapethissite.com/pages/forms/](https://www.scrapethissite.com/pages/forms/)

First, as with any web scraping task, we need to inspect the page to understand the structure of the page.

We'll start by scraping the first page:

In [ ]:
r = requests.get("https://www.scrapethissite.com/pages/forms/")

In [ ]:
r.status_code

In [ ]:
soup = BeautifulSoup(r.text, "html.parser")

Let's extract the table header first:

In [ ]:
header = soup.find('tr')

In [ ]:
header

In [ ]:
headers = [th.text.strip() for th in header.find_all('th')]

In [ ]:
headers

Let's find all the rows on first page:

In [ ]:
teams = soup.find_all('tr', 'team')

In [ ]:
teams[0]

In [ ]:
for col in teams[0].find_all('td'):
    print(col.text.strip())

We can combine that with the header:

In [ ]:
row_dict = {}
for header, col in zip(headers, teams[0].find_all('td')):
    row_dict[header] = col.text.strip()

In [ ]:
row_dict

> Repeat the operation above for all the teams (rows) using a `for` loop, and append each row_dict to a list called `rows`.

If you press on any page number (button) in the bottom, you will see the URL change, for example pressing on page 2:

`https://www.scrapethissite.com/pages/forms/?page_num=2`

This is because the page number is a parameter in the URL. We can use this to our advantage to scrape all the pages. We can use a `for loop` to iterate through all the pages and scrape the data from each page.

In [ ]:

def parse_and_extract_rows(response_text: str):
    """
    Parse the HTML and extract table rows from the response object's text.
    
    Args:
        response_text (str): The HTML text from the response object.
        
    Returns:
        An iterator of dictionaries with the data from the current page.
    """
    soup = BeautifulSoup(response_text, "html.parser")
    header = soup.find('tr')
    headers = [th.text.strip() for th in header.find_all('th')]
    teams = soup.find_all('tr', 'team')
    for team in teams:
        row_dict = {}
        for header, col in zip(headers, team.find_all('td')):
            row_dict[header] = col.text.strip()
        yield row_dict

As there are 24 pages, we need to iterate through all of them to get all the data we want. We will use a for loop to do this. We will also use the `time.sleep()` function to pause and make sure we don't overload the server with requests.

In [ ]:
import time

In [ ]:
rows = []
for page in range(1, 25):
    r = requests.get(f"https://www.scrapethissite.com/pages/forms/?page_num={page}")
    for row_dict in parse_and_extract_rows(r.text):
        rows.append(row_dict)
    # pause for 1 second between requests
    time.sleep(1)

In [ ]:
len(rows)

In [ ]:
rows[0]

In [ ]:
rows[-1]

Let's convert the list of dictionaries into a dataframe to perform further analysis.

In [ ]:
df = pd.DataFrame(rows)

 Cast the correct dtypes for the columns, we'll use pandas [nullable](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes) integer (`pd.Int64Dtype`) and float (`pd.Float64Dtype`) type for the numeric columns.

 Below, `int` is an alias for `np.int64`, while `Int64` represents `pd.Int64Dtype`.

In [ ]:
df = df.replace('', pd.NA).astype({'Year': int, 'Wins': 'Int64', 'Losses': 'Int64', 'OT Losses': 'Int64', 
                                   'Goals For (GF)': 'Int64', 'Goals Against (GA)': 'Int64', 
                                   '+ / -': 'Int64', 'Win %': float})

In [ ]:
df

> 1. Find the team that has the most number of Wins for the year.
> 
> 2. Find the team that has the most number of Losses for the year.
> 
> 3. Find the team that has the lowest Win % for the year.
> 
> 4. Find the team that has the most number of Wins for all years.